## Adding parameters to the data model
**ema20** = exponential moving average 20 days.
**ema50** = exponential moving average 50 days.
**openHigher** = If Open > (1,01 * Previous day Close) then 1 else 0.
**strongClose** = function=(High + Low) / 2. If Close > 'function' then 1 else 0. Maybe not necessary. Lets try model with and without the strongClose parameter.
**averageVolume** = Average volume from past 200 days. 200 simple moving average for volume.
**strongVolume** = If Volume > (2 * averageVolume) then 1 else 0.
**strongVolume6MoPrior** = If strongVolume is equal to 1 more than 3 times in the past 126 days then 1 else 0.
**strongVolumeAfterFiling** = If strongVolume is equal to 1 more 2 times in the following 5 days after financialRelease = 1 then 1 else 0.
**accVolume** = If strongVolume = 1 and strongVolume6MoPrior = 1 and strongVolumeAfterFiling = 1 then 1 else 0.
**uptrend** = If ema50 > (ema50 50 days prior) then 1 else 0.

In [5]:
import requests
import pandas as pd
# from datetime import datetime, timedelta, date
import datetime
import time
from polygon import RESTClient
import logging
import signal
import sys
import pickle
import lz4.frame  # type: ignore
import concurrent.futures
import os
import pandas as pd
import numpy as np
import glob
import nbimporter
from readRawAggs import main

In [ ]:
# # Define the path to your existing CSV file
# csv_file_path = r"C:\Users\SamuliMustonen\Documents\Ready Solutions\Docs\StockTrading\Data\ntnx_data_raw.csv"
# # Load the CSV file into a DataFrame
# df = pd.read_csv(csv_file_path, delimiter=';', header=0)

In [ ]:
# Function to calculate EMA
def calculate_ema(data, window):
    """Calculate the Exponential Moving Average (EMA) for a given window."""
    # Calculate EMA using ewm() and handle insufficient data gracefully
    ema = data['close'].ewm(span=window, adjust=False).mean()
    
    # Replace initial values with NaN for insufficient data
    if len(data) < window:
        ema[:window] = None  # Set initial window values to None (NaN)
    
    return ema

In [12]:
# Use main() from readRawAggs

def myfunction():
    df = main()
    if df is not None:
        print(df.head())
myfunction()

In [ ]:
# Use main() from readRawAggs
df = main()

# Calculate EMA20 and EMA50
df['ema20'] = calculate_ema(df, 20)
df['ema50'] = calculate_ema(df, 50)
# Round ema20 and ema50 to 2 decimal places
df['ema20'] = df['ema20'].round(2)
df['ema50'] = df['ema50'].round(2)

# **openHigher** = If Open > (1.01 * Previous day Close) then 1 else 0
df['openHigher'] = (df['open'] > (1.01 * df['close'].shift(1))).astype(int)

# **strongClose** = (High + Low) / 2. If Close > 'function' then 1 else 0
df['function'] = (df['high'] + df['low']) / 2
df['strongClose'] = (df['close'] > df['function']).astype(int)

# Convert Volume to integer format
df['volume'] = df['volume'].astype(int)  # Convert volume to integer

# **averageVolume** = Average volume from past 200 days (SMA for volume)
df['averageVolume'] = df['volume'].rolling(window=200).mean()
df['averageVolume'] = df['averageVolume'].astype(int)

# **strongVolume** = If Volume > (2 * averageVolume) then 1 else 0
df['strongVolume'] = (df['volume'] > 2 * df['averageVolume']).astype(int)

# **strongVolume6MoPrior** = If strongVolume is 1 more than 3 times in the past 126 days then 1 else 0
df['strongVolume6MoPrior'] = df['strongVolume'].rolling(window=126).sum().shift(1)
df['strongVolume6MoPrior'] = (df['strongVolume6MoPrior'] > 3).astype(int)

# **strongVolumeAfterFiling** = If strongVolume is 1 more than 2 times in the 5 days after financialRelease = 1 then 1 else 0
# df['strongVolumeAfterFiling'] = 0
# for i in range(len(df) - 5):
#     if df.loc[i, 'financialRelease'] == 1:
#         if df.loc[i+1:i+5, 'strongVolume'].sum() > 2:
#             df.loc[i+1:i+5, 'strongVolumeAfterFiling'] = 1

# **accVolume** = If strongVolume = 1, strongVolume6MoPrior = 1, and strongVolumeAfterFiling = 1 then 1 else 0
df['accVolume'] = ((df['strongVolume'] == 1) & 
                   (df['strongVolume6MoPrior'] == 1)) 
                #    (df['strongVolumeAfterFiling'] == 1)).astype(int)

# **uptrend** = If ema50 > (ema50 50 days prior) then 1 else 0
df['uptrend'] = (df['ema50'] > df['ema50'].shift(50)).astype(int)

# Drop the 'function' column as it's intermediate
df.drop(columns=['function'], inplace=True)

# Save the updated DataFrame to a new CSV file with a timestamp
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
new_csv_file_path = f"C:/Users/SamuliMustonen/Documents/Ready Solutions/Docs/StockTrading/Data/data_model_{timestamp}.csv"
df.to_csv(new_csv_file_path, index=False)

print(f"Updated data with new columns saved to {new_csv_file_path}")
